In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [3]:

import dask.array as da
import numpy as np
import pandas as pd
from utils import dasker, config as util_config
from dask_ml.preprocessing import StandardScaler


from keras import models, layers
from keras.optimizers import Adam, RMSprop
from keras.layers import Dense
from keras.models import Sequential
from keras.backend import clear_session

import warnings

import optuna
import dask_optuna
from ml.preprocess.data import SingletonDataSet
# from tensorflow.keras.optimizers import RMSprop

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
import dask.dataframe as ddf
from dask import distributed

In [4]:
pwd

'/mnt/c/Users/rwmas/GitHub/xai/xai_api/app/notebooks'

In [5]:
# df = pd.read_csv('../test/data/20220319_covid_merge_processed.csv', sep=",")

In [ ]:
import pandas as pd
def get_dataset():
    df = pd.read_csv('../test/data/20220319_covid_merge_processed.csv', sep=",")

    dataset = SingletonDataSet()
    dataset.load_data(df, 0, 0, 'regression')
    X, y = dataset.get_data()
    return X, y

    


In [6]:
import pandas as pd
def get_dataset():
    df = pd.read_csv('../test/data/20220319_covid_merge_processed.csv', sep=",")

    dataset = SingletonDataSet()
    dataset.load_data(df, 0, 0, 'regression')
    X, y = dataset.get_data()
    return X, y

    


In [7]:

def build_shallow_model(trial, input_dim, lr, loss, pred_type):

    model = models.Sequential()
    # Input layer

    n_layers = trial.suggest_int("n_layers", 1, 3)
    hidden_units = []

    model.add(layers.BatchNormalization())

    for i in range(n_layers):
        n_units = trial.suggest_int("n_units_l{}".format(i), 1, 128)
        # hidden_units.append(n_units)
        # Input layer
        model.add(layers.Dense(n_units, name = "dense_hidden_"+str(i), kernel_initializer='normal', activation='relu'))

    #model.add(layers.Dense(32, name = "dense_hidden", input_dim=input_dim, kernel_initializer='normal', activation='relu'))
        
    # model.add(layers.Dense(8, name = "out", activation='relu'))
    
    # Output layer
    if pred_type == 'classification':
        model.add(layers.Dense(1, activation='sigmoid'))
    else:
        model.add(layers.Dense(1, activation='relu'))

    # Compile a model
    model.compile(loss=loss, optimizer=Adam(lr), metrics=['accuracy'])
    return model



In [ ]:
from dask_ml.model_selection import train_test_split
def objective_dask_array(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    scores = []

    df_X, df_y = get_dataset()

    df_X = StandardScaler().fit_transform(df_X)


    X = ddf.from_pandas(df_X, npartitions=no_of_workers)
    y = ddf.from_pandas(df_y, npartitions=no_of_workers)

    X = X.to_dask_array(lengths=True)
    y = y.to_dask_array(lengths=True)


    X_base, X_val, y_base, y_val = train_test_split(X, y, random_state=0, shuffle=True)
    
    input_shape = X_base.shape[1]

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)

    loss = 'mean_squared_error'

    model = build_shallow_model(trial, input_shape, learning_rate, loss, 'regression')


    if pred_type == 'classification':
        cv = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
    else:
        cv = KFold(n_splits=5, random_state=None, shuffle=False)



    for train_idx, test_idx in cv.split(X_base, y_base):
        X_train, y_train = X_base[train_idx], y_base[train_idx]
        X_test, y_test = X_base[test_idx], y_base[test_idx]

        model.fit(
                X_train,
                y_train,
                # validation_data=(X_val, y_val),        
                shuffle=True,
                batch_size=BATCHSIZE,
                epochs=EPOCHS,
                verbose=False,
            )

        # Evaluate the model accuracy on the validation set.
        # score = model.evaluate(X_val, y_val, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=0)

        scores.append(score[1])

        y_pred = model.predict(X_test)

        error = mean_squared_error(y_test, y_pred)
                
    print(np.mean(error))




    

    # Evaluate the model accuracy on the validation set.
    #score = model.evaluate(X_val, y_val, verbose=0)
    # return score[1]

In [85]:

from sklearn.model_selection import train_test_split

def objective_cv(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    scores = []

    df_X, df_y = get_dataset()

    df_X = StandardScaler().fit_transform(df_X)

    X_base, X_val, y_base, y_val = train_test_split(df_X, df_y, random_state=0, shuffle=True)
    
    input_shape = X_base.shape[1]

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)

    loss = 'mean_absolute_error'

    model = build_shallow_model(trial, input_shape, learning_rate, loss, 'regression')

    if pred_type == 'classification':
        cv = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)
    else:
        cv = KFold(n_splits=5, random_state=None, shuffle=False)



    for train_idx, test_idx in cv.split(X_base, y_base):
        X_train, X_test = X_base.loc[X_base.index[train_idx]], X_base.loc[X_base.index[test_idx]]
        y_train, y_test = y_base.loc[y_base.index[train_idx]], y_base.loc[y_base.index[test_idx]]
        
        model.fit(
                X_train,
                y_train,
                # validation_data=(X_val, y_val),        
                shuffle=True,
                batch_size=BATCHSIZE,
                epochs=EPOCHS,
                verbose=False,
            )

        # Evaluate the model accuracy on the validation set.
        # score = model.evaluate(X_val, y_val, verbose=0)
        # score = model.evaluate(X_test, y_test, verbose=0)
        
        y_pred = model.predict(X_test)

        error = np.round_(mean_squared_error(y_test, y_pred), decimals=2, out=None)
                
        scores.append(error)

        # scores.append(score[1])
                
    mean_score = np.mean(scores)

    print('mean score:' + str(mean_score))


    # Evaluate the model accuracy on the validation set.
    #score = model.evaluate(X_val, y_val, verbose=0)
    return mean_score

In [22]:

BATCHSIZE = 128
EPOCHS = 1000
N_TRIALS = 50
no_of_workers = 4

pred_type = "regression"

model_path = "/mnt/c/Users/rwmas/GitHub/xai/xai_api/app/ml/models/saved/base/brisk_neural_net/"
# CLASSES = 10

In [37]:

from sklearn.model_selection import train_test_split

def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    scores = []

    df_X, df_y = get_dataset()

    df_X = StandardScaler().fit_transform(df_X)

    X_base, X_val, y_base, y_val = train_test_split(df_X, df_y, random_state=0, shuffle=True)
    
    input_shape = X_base.shape[1]

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)

    bs = trial.suggest_categorical("batch_size", [16, 32, 64, 128, 512, 1024, 2048])

    loss = 'mean_squared_error'

    model = build_shallow_model(trial, input_shape, learning_rate, loss, 'regression')

        
    model.fit(
            X_base,
            y_base,
            validation_data=(X_val, y_val),
            shuffle=True,
            batch_size=bs,
            epochs=EPOCHS,
            verbose=False,
        )
    
    y_pred = model.predict(X_val)

    err = np.round_(mean_squared_error(y_val, y_pred), decimals=2, out=None)
    
    print(err)

    # Evaluate the model accuracy on the validation set.
    #score = model.evaluate(X_val, y_val, verbose=0)
    return err


In [38]:
from dask.distributed import Client
import joblib

# with Client() as client:
client = dasker.get_dask_client()
print(f"Dask dashboard is available at {client.dashboard_link}")

storage = dask_optuna.DaskStorage()
study = optuna.create_study(storage=storage, direction="minimize")

with joblib.parallel_backend("dask"):
    study.optimize(objective, n_trials=10, n_jobs=-1)

print("Number of trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-09-15 09:00:34,081] A new study created in memory with name: no-name-410f11e5-eacc-4cc0-8b26-796dc213272d


Dask dashboard is available at http://127.0.0.1:8787/status
2/2 [==============================] - 0s 2ms/step
4159987.17


[I 2022-09-15 09:01:22,368] Trial 0 finished with value: 4159987.17 and parameters: {'learning_rate': 1.2253155021127636e-05, 'batch_size': 512, 'n_layers': 2, 'n_units_l0': 4, 'n_units_l1': 39}. Best is trial 0 with value: 4159987.17.


2/2 [==============================] - 0s 2ms/step
1130238.2


[I 2022-09-15 09:01:24,022] Trial 1 finished with value: 1130238.2 and parameters: {'learning_rate': 0.0006761863357037314, 'batch_size': 512, 'n_layers': 3, 'n_units_l0': 19, 'n_units_l1': 33, 'n_units_l2': 53}. Best is trial 1 with value: 1130238.2.


...

[I 2022-09-15 09:01:44,184] Trial 5 finished with value: 1311320.06 and parameters: {'learning_rate': 0.03922372632682, 'batch_size': 32, 'n_layers': 3, 'n_units_l0': 26, 'n_units_l1': 86, 'n_units_l2': 61}. Best is trial 7 with value: 1020612.09.


2/2 [==============================] - 0s 3ms/step
1311320.06
2/2 [==============================] - 0s 1ms/step
1070602.17


[I 2022-09-15 09:01:44,602] Trial 3 finished with value: 1070602.17 and parameters: {'learning_rate': 0.00013807148520469757, 'batch_size': 16, 'n_layers': 3, 'n_units_l0': 50, 'n_units_l1': 6, 'n_units_l2': 59}. Best is trial 7 with value: 1020612.09.
[I 2022-09-15 09:01:49,610] Trial 6 finished with value: 3896770.0 and parameters: {'learning_rate': 1.4680001680461255e-05, 'batch_size': 16, 'n_layers': 2, 'n_units_l0': 17, 'n_units_l1': 46}. Best is trial 7 with value: 1020612.09.


2/2 [==============================] - 0s 2ms/step
3896770.0
2/2 [==============================] - 0s 1ms/step
1595869.44
Number of trials: 10
Best trial:
  Value: 1020612.09
  Params: 
    learning_rate: 0.021259734643756672
    batch_size: 64
    n_layers: 3
    n_units_l0: 101
    n_units_l1: 62
    n_units_l2: 30


[I 2022-09-15 09:02:00,018] Trial 8 finished with value: 1595869.44 and parameters: {'learning_rate': 0.00033344001803178435, 'batch_size': 1024, 'n_layers': 2, 'n_units_l0': 48, 'n_units_l1': 11}. Best is trial 7 with value: 1020612.09.


In [39]:
pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  0
  Number of complete trials:  10


In [40]:
trial.params

{'learning_rate': 0.021259734643756672,
 'batch_size': 64,
 'n_layers': 3,
 'n_units_l0': 101,
 'n_units_l1': 62,
 'n_units_l2': 30}

In [41]:
import fnmatch

loss = 'mean_squared_error'

learning_rate = trial.params['learning_rate']

batch_size = trial.params['batch_size']

n_layers = trial.params['n_layers']
units = []
for param in trial.params:
    if fnmatch.fnmatch(param, 'n_units*'):
        # print(param)
        units.append(param)  

In [42]:


best_model = models.Sequential()
# Input layer

best_model.add(layers.BatchNormalization())

for i in range(n_layers):
    best_model.add(layers.Dense(trial.params[units[i]], name = "dense_hidden_"+str(i), kernel_initializer='normal', activation='relu'))

best_model.add(layers.Dense(1, activation='relu'))

# Compile a model
best_model.compile(loss=loss, optimizer=Adam(learning_rate), metrics=['accuracy'])



In [43]:
df_X, df_y = get_dataset()

# df_X = StandardScaler().fit_transform(df_X)

X_base, X_val, y_base, y_val = train_test_split(df_X, df_y, random_state=0, shuffle=True)

input_shape = X_val.shape[1]


In [44]:
    
best_model.fit(
        X_base,
        y_base,
        shuffle=True,
        batch_size=batch_size,
        epochs=300,
        verbose=False,
    )



In [45]:
best_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_1 (Batc  (None, 41)               164       
 hNormalization)                                                 
                                                                 
 dense_hidden_0 (Dense)      (None, 101)               4242      
                                                                 
 dense_hidden_1 (Dense)      (None, 62)                6324      
                                                                 
 dense_hidden_2 (Dense)      (None, 30)                1890      
                                                                 
 dense_1 (Dense)             (None, 1)                 31        
                                                                 
Total params: 12,651
Trainable params: 12,569
Non-trainable params: 82
_________________________________________________

In [46]:
x_val_pred = best_model.predict(X_val)

2/2 [==============================] - 0s 2ms/step


In [47]:
#err = 
mean_squared_error(y_val, x_val_pred)
    
# print(err)

4321093.345267444

In [48]:
score = best_model.evaluate(X_val, y_val, batch_size = batch_size, verbose=0)
score

[4321094.0, 0.0]

In [49]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
best_model.save(model_path)